In [ ]:
import io, re
from datetime import datetime
from pypdf import PdfReader
from docx import Document
import ipywidgets as w
from IPython.display import display, clear_output


In [ ]:
titulo = w.HTML("<h2>Extrair e-mails de PDFs</h2>")
descricao = w.HTML("""
<p>Envie um ou mais PDFs, clique em <b>Processar</b> e baixe o arquivo <code>.docx</code>.</p>
""")
upload = w.FileUpload(accept='.pdf', multiple=True)
btn = w.Button(description='Processar', button_style='primary', icon='cogs')
log = w.Output()

def extrair_emails_pdf_bytes(pdf_bytes: bytes):
    with io.BytesIO(pdf_bytes) as f:
        reader = PdfReader(f)
        texto = []
        for page in reader.pages:
            t = page.extract_text() or ""
            texto.append(t)
        texto = "\n".join(texto)
    return re.findall(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', texto)

def gerar_docx(emails_unicos, saida):
    doc = Document()
    doc.add_heading('Lista de E-mails Encontrados', level=1)
    doc.add_paragraph(f'Gerado em: {datetime.now().strftime("%d/%m/%Y %H:%M:%S")}')
    doc.add_paragraph('')
    if emails_unicos:
        for e in sorted(set(emails_unicos), key=str.lower):
            doc.add_paragraph(e)
    else:
        doc.add_paragraph('Nenhum e-mail encontrado.')
    doc.save(saida)

def on_process(_):
    with log:
        clear_output()
        if not upload.value:
            print("⚠️ Nenhum arquivo PDF enviado.")
            return
        print(f"📄 Arquivos recebidos: {len(upload.value)}\n")
        todos = []
        for nome, meta in upload.value.items():
            real_name = meta.get("name", nome)
            content = meta.get("content")
            print(f"• {real_name} ...", end=" ")
            try:
                emails = extrair_emails_pdf_bytes(content)
                todos.extend(emails)
                print(f"{len(emails)} e-mail(s)")
            except Exception as e:
                print(f"Erro: {e}")
        unicos = sorted(set(todos))
        print(f"\n📬 Total únicos: {len(unicos)}")
        saida = "emails_encontrados.docx"
        gerar_docx(unicos, saida)
        print(f"\n✅ Pronto! Baixe: {saida}")
        print("\n💾 Para baixar: clique no ícone de pasta no topo esquerdo do Binder → clique no arquivo .docx.")

btn.on_click(on_process)
display(w.VBox([titulo, descricao, upload, btn, log]))
